In [1]:
from numpy import loadtxt
import numpy as np
import math
import matplotlib.pyplot as plt
import pprint
import pandas as pd
from IPython.display import display, HTML

In [2]:
#Loading Data Files
#Features
trainingData = loadtxt("/Users/jayavardhanreddy/Python_Code/ML/HW4/DataFiles/trainingData.txt", delimiter=",",unpack=False,dtype=np.float128)
testData = loadtxt("/Users/jayavardhanreddy/Python_Code/ML/HW4/DataFiles/testData.txt", delimiter=",", unpack=False,dtype=np.float128)
validationData = loadtxt("/Users/jayavardhanreddy/Python_Code/ML/HW4/DataFiles/validationData.txt", delimiter=",",unpack=False,dtype=np.float128)

#Labels
trainingLabels = loadtxt("/Users/jayavardhanreddy/Python_Code/ML/HW4/DataFiles/trainingLabels.txt",unpack=False,dtype=np.float128)
testLabels = loadtxt("/Users/jayavardhanreddy/Python_Code/ML/HW4/DataFiles/testLabels.txt",unpack=False,dtype=np.float128)
validationLabels = loadtxt("/Users/jayavardhanreddy/Python_Code/ML/HW4/DataFiles/validationLabels.txt",unpack=False,dtype=np.float128)

In [3]:
#Normalization
trainingData_norm = (trainingData/128.0)-1.0
testData_norm = (testData/128.0)-1.0
validationData_norm = (validationData/128.0)-1.0

In [4]:
#Function to calculate gradient
def calculate_k(x,y):
    return np.power((np.dot(x,y.T)/784+1),3)

k=calculate_k(trainingData_norm,trainingData_norm)

In [5]:
def calculate_gradient_a(a,k_a_b,lamda):
    y=trainingLabels
    N=len(y)
    gradient=np.zeros((N,),dtype=np.float128)
    for yp,p in zip(y,range(N)):
        value=1-(yp*(k_a_b[p]))
        if value>=0:
            gradient+=-1.0*(yp)*(k[p].T)
            
    gradient/=N
    gradient=np.reshape(gradient,(N,1))
    gradient+=((lamda/2.0)*np.dot((k+k.T),a))
    return gradient

def calculate_gradient_b(a,k_a_b,lamda):
    y=trainingLabels
    gradient=0
    N=len(y)
    for yp,p in zip(y,range(N)):
        value=1-(yp*(k_a_b[p]))
        if value>=0:
            gradient+=-1.0*yp
    gradient/=N
    return gradient

#Main Function for Gradient Descent
def gradient_descent(T,learning_rate,lamda):
    a=np.zeros((len(trainingData_norm),1), dtype=np.float128)
    b=0
    for t in range(1,T+1):
        k_a_b=np.dot(k,a)+b
        gradient_a=calculate_gradient_a(a,k_a_b,lamda)
        gradient_b=calculate_gradient_b(a,k_a_b,lamda)
        a-=(learning_rate*gradient_a)
        b-=(learning_rate*gradient_b)
    return a,b

In [6]:
learning_rate=0.00001
lamda=0.001
T=500
a,b=gradient_descent(T,learning_rate,lamda)
print(a)
print(b)

[[ 3.9972937e-05]
 [ 0.00012790565]
 [ 0.00075335095]
 ..., 
 [ 0.00034678965]
 [-0.00046754469]
 [-0.00068844037]]
-3.08482069756e-05


In [7]:
#Function to calculate Risk
def calculate_error(a,b,x,y):
    x_train=trainingData_norm
    N=len(y)
    k_=calculate_k_linear(x,x_train)
    prediction=np.sum(np.multiply(k_,a),axis=1)+b
    prediction[prediction <= 0] = -1.0
    prediction[prediction > 0] = 1.0
    wrong_classification=len([a for a,b in zip(prediction,y) if a!=b])
    error=wrong_classification*100.0/N
    return error

In [8]:
print('calculating Validation risk for learning_rate:',learning_rate,'and lamda:',lamda)
print(calculate_error(a,b,trainingData_norm,trainingLabels))

calculating Validation risk for learning_rate: 1e-05 and lamda: 0.001


NameError: name 'calculate_k_linear' is not defined

In [ ]:
learning_rate=0.00000000001
lamda=0.00000001
T=50
a,b=gradient_descent(T,learning_rate,lamda)
print(a)
print(b)

In [ ]:
print('calculating Validation risk for learning_rate:',learning_rate,'and lamda:',lamda)
print(calculate_error(a,b,trainingData_norm,trainingLabels))

In [ ]:
#10 different Learning rates
#Training and Validation risk for each learning rate
print('Training and Validation risk for 54 (learning rates,Lambda) combinations')
print('Learning rates:0.000001,0.000005,0.00001,0.00005,0.0001,0.005,0.001,0.01,1')
print('Lambda:0.0001,0.001,0.01,0.1,0,1')

validation_errors=[]
training_errors=[]
learning_rates=[]
lambdas=[]
T=10
for learning_rate in [0.000001,0.000005,0.00001,0.00005,0.0001,0.005,0.001,0.01,1]:
    for lamda in [0.0001,0.001,0.01,0.1,0,1]:
        a,b=gradient_descent(T,learning_rate,lamda)
        learning_rates.append(learning_rate)
        lambdas.append(lamda)
        
        validation_error=calculate_error(a,b,validationData_norm,validationLabels)
        validation_errors.append(validation_error)
        
        training_error=calculate_error(a,b,trainingData_norm,trainingLabels)
        training_errors.append(training_error)

In [ ]:
df=pd.DataFrame(columns=['Learning_rate','Lambda','Training_Error','Validation_Error'])
df['Learning_rate']=learning_rates
df['Lambda']=lambdas
df['Training_Error']=training_errors
df['Validation_Error']=validation_errors
#display(df)
#print "Dataframe 2:"
HTML(df.to_html())

In [ ]:
#Learning_rate which yields least validation_error
minimum_validation_error=min(validation_errors)
index_of_minimum=validation_errors.index(minimum_validation_error)
learning_rate,lamda=learning_rates[index_of_minimum],lambdas[index_of_minimum]
print("learning_rate for least validation risk",learning_rate)
print("lambda for least validation risk",lamda)
print('Validation Error for learning rate:',learning_rate,'and lambda:',lamda,'is:',minimum_validation_error)

In [ ]:
#Function used to generate list of empirical risk for iterations 1 to T
def Empiricial_risk_vs_iterations(T,learning_rate,lamda):
    a=np.zeros((len(trainingData_norm),1), dtype=np.float128)
    b=0
    list_empirical_risk=[]
    for t in range(1,T+1):
        k_a_b=np.dot(k,a)+b
        gradient_a=calculate_gradient_a(a,k_a_b,lamda)
        gradient_b=calculate_gradient_b(a,k_a_b,lamda)
        a-=(learning_rate*gradient_a)
        b-=(learning_rate*gradient_b)
        
        empirical_risk=calculate_error(a,b,trainingData_norm,trainingLabels)
        list_empirical_risk.append(empirical_risk)
    return list_empirical_risk

#list of empirical risk values, for iterations from 1 to 1000
T=500
list_empirical_risk=Empiricial_risk_vs_iterations(T,learning_rate,lamda)

In [ ]:
#Plot Empirical Risk vs Iterations
plt.figure(figsize=(10,10))
plt.plot(range(1,501), list_empirical_risk, 'r--')
plt.ylabel('Empiricial risk')
plt.xlabel('Iterations')
plt.title(' Empirical Risk vs Iterations')
plt.show()

#Find number of iterations which gives minimum risk 
number_of_iterations=list_empirical_risk.index(min(list_empirical_risk))+1
print("Iteration which produces least empirical risk:",number_of_iterations)

In [ ]:
#Final Model
#Best model using the learning rate and number_of_iterations found
a,b=gradient_descent(number_of_iterations,learning_rate,lamda)
test_error=calculate_error(a,b,testData_norm,testLabels)
print("best model: learning rate:",learning_rate,'and lamda:',lamda,' for iterations:',number_of_iterations)
print('Test Risk of the best model:',test_error)

print('\nNote:')
print('Test Risk:',test_error, 'and Validation Risk:',minimum_validation_error,'are close')
print('Empirical Risk reduces as iterations increase as expected')

In [ ]:
#Validation Accuracy vs Test Accuracy

#Function to calculate Risk
def calculate_accuracy(a,b,x,y):
    x_train=trainingData_norm
    N=len(y)
    k_=calculate_k(x,x_train)
    prediction=np.sum(np.multiply(k_,a),axis=1)+b
    prediction[prediction <= 0] = -1.0
    prediction[prediction > 0] = 1.0
    correct_classification=len([a for a,b in zip(prediction,y) if a==b])
    accuracy=correct_classification*100.0/N
    return accuracy

a,b=gradient_descent(number_of_iterations,learning_rate,lamda)
#test_accuracy=calculate_accuracy(w,testData,testLabels)
validation_accuracy=calculate_accuracy(a,b,validationData_norm,validationLabels)
training_accuracy=calculate_accuracy(a,b,trainingData_norm,trainingLabels)

#print('training_accuracy',training_accuracy)
print('validation_accuracy',validation_accuracy)
print('test_accuracy',test_accuracy)

print('Note:')
print('Validation Accuracy and Test Accuracy are close')